In [3]:
!sudo -v ; curl https://rclone.org/install.sh | sudo bash
!git clone https:/@github.com/berstearns/pn.git
!cp /content/pn/rclone/rclone.conf /root/.config/rclone/rclone.conf
!pip install rclone-python
!rclone config file

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4734  100  4734    0     0  10565      0 --:--:-- --:--:-- --:--:-- 10566

The latest version of rclone rclone v1.69.0 is already installed.

fatal: destination path 'pn' already exists and is not an empty directory.
Configuration file is stored at:
/root/.config/rclone/rclone.conf


In [4]:
from rclone_python import rclone

In [5]:
if not rclone.get_remotes():
  input("NO remotes to get data from")
else:
  print(rclone.get_remotes())

['insight-gdrive:', 'berstearns-gdrive:', 'bernardostearns-gdrive:']


In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
import pandas as pd
from itertools import product
from sklearn.metrics import classification_report
import warnings
from collections import defaultdict
import os
from rclone_python import rclone
import pickle
warnings.filterwarnings('ignore')


In [37]:
notebook_dir = os.path.abspath("")
local_base_dir = os.path.dirname(notebook_dir)
local_experiment_data_folder = os.path.join(local_base_dir,"datasets", "NLP4CALL_2025_experiment","experiments_data")


def remote_expected(config):
    """
    Check if the configuration expects remote storage for inputs or outputs.
    Returns a tuple:
    - A boolean indicating if remote storage is expected.
    - The remote base directory (if remote storage is expected), otherwise None.
    """
    input_remote = config["storage"]["inputs"]["from"] == "rclone"
    output_remote = config["storage"]["outputs"]["from"] == "rclone"

    # Determine if remote storage is expected
    is_remote = input_remote or output_remote

    # Determine the remote base directory
    remote_base_dir = None
    if input_remote:
        remote_base_dir = config["storage"]["inputs"].get("remote_base_dir")
    if output_remote:
        remote_base_dir = config["storage"]["outputs"].get("remote_base_dir")

    # If both input and output are remote, ensure they have the same base directory
    if input_remote and output_remote:
        input_dir = config["storage"]["inputs"].get("remote_base_dir")
        output_dir = config["storage"]["outputs"].get("remote_base_dir")
        if input_dir != output_dir:
            raise ValueError("Input and output remote base directories must be the same when both are remote.")
    remote_flags = {
        "input": input_remote,
        "output": output_remote,
    }
    return is_remote, remote_base_dir, remote_flags


config_local_local = {
    "storage": {
        "inputs": {
            "from": "local"
        },
        "outputs": {
            "from": "local"
        },
    },
    "text_column": "text",
    "column_mapping": {
        "remainder_efcamdat": {
            "text":"",
            "CEFR":"?"
        },
        "andrew100k": {
            "text": "text",
            "CEFR":"cefr_level"
        }
    },
    "perplexity_models" : [
        "gpt2"
    ],
    "base_columns": ['writing_id', 'cefr_level','text', 'l1' ],
}

config_local_rclone = {
    "storage": {
        "inputs": {
            "from": "local"
        },
        "outputs": {
            "from": "rclone",
            "remote_base_dir": "insight-gdrive:/phd-experimental-data"
        },
    },
    "text_column": "text",
    "column_mapping": {
        "remainder_efcamdat": {
            "text":"",
            "CEFR":"?"
        },
        "andrew100k": {
            "text": "text",
            "CEFR":"cefr_level"
        }
    },
    "perplexity_models" : [
        "gpt2"
    ],
    "base_columns": ['writing_id', 'cefr_level','text', 'l1' ],
}

config_rclone_rclone = {
    "storage": {
        "inputs": {
            "from": "rclone",
            "remote_base_dir": "insight-gdrive:/phd-experimental-data"
        },
        "outputs": {
            "from": "rclone",
            "remote_base_dir": "insight-gdrive:/phd-experimental-data"
        },
    },
    "text_column": "text",
    "column_mapping": {
        "remainder_efcamdat": {
            "text":"",
            "CEFR":"?"
        },
        "andrew100k": {
            "text": "text",
            "CEFR":"cefr_level"
        }
    },
    "perplexity_models" : [
        "gpt2"
    ],
    "base_columns": ['writing_id', 'cefr_level','text', 'l1' ],
}


config = config_rclone_rclone


is_remoted_expected, remote_base_dir, remote_flags = remote_expected(config)
config["remote_flags"] = remote_flags
if is_remoted_expected:
   remote_experiment_data_folder = os.path.join(remote_base_dir, "NLP4CALL_2025_experiment","experiments_data")


andrew_100k_train_fe_local_fp  = os.path.join(local_experiment_data_folder, "andrew100k-train-fe.csv")
if remote_flags["input"] == True:
    # expects a remote + the same folder structure as local folder structure
    # efcamdat_100k_with_text_and_measures.csv
    if not os.path.exists(andrew_100k_train_fe_local_fp):
        andrew_100k_train_fe_remote_fp = os.path.join(remote_experiment_data_folder, "andrew100k-train-fe.csv","andrew100k-train-fe.csv")
        os.makedirs(local_experiment_data_folder,exist_ok=True)
        rclone.copyto(andrew_100k_train_fe_remote_fp, andrew_100k_train_fe_local_fp)

if remote_flags["output"] == False:
    pass
    #config["train_output_fp"] =  os.path.join(local_experiment_data_folder, "andrew100k-train-fe.csv")
    #config["test_output_fp"] =  os.path.join(local_experiment_data_folder, "andrew100k-test-fe.csv")
elif remote_flags["output"] == True:
    pass
    # expects a remote + the same folder structure as local folder structure
    config["cv_train_output_fp"] =  os.path.join(remote_experiment_data_folder, "andrew100k-train-fe-cv.pickle")
    #config["test_output_fp"] =  os.path.join(remote_experiment_data_folder, "andrew100k-test-fe.csv")
else:
    raise Exception("unexpected output remote flag")



In [16]:
config

{'storage': {'inputs': {'from': 'rclone',
   'remote_base_dir': 'insight-gdrive:/phd-experimental-data'},
  'outputs': {'from': 'rclone',
   'remote_base_dir': 'insight-gdrive:/phd-experimental-data'}},
 'text_column': 'text',
 'column_mapping': {'remainder_efcamdat': {'text': '', 'CEFR': '?'},
  'andrew100k': {'text': 'text', 'CEFR': 'cefr_level'}},
 'perplexity_models': ['gpt2'],
 'base_columns': ['writing_id', 'cefr_level', 'text', 'l1'],
 'remote_flags': {'input': True, 'output': True}}

In [17]:
idx_to_class_ = lambda v: {
         0: "A1",
         1: "A2",
         2: "B1",
         3: "B2",
         4: "C1",
         5: "C1",
}.get(v, None)
label_to_idx_ = lambda v: {
         "A1": 0,
         "A2": 1,
         "B1": 2,
         "B2": 3,
         "C1": 4,
         "C2": 4,
}.get(v, None)

In [10]:
andrew100k_train_fe_df=pd.read_csv(andrew_100k_train_fe_local_fp)

In [18]:
andrew100k_train_fe_df.head()

,writing_id,cefr_level,text,l1,measures.andrew.collocations.text_level.ratio_num_token,measures.andrew.collocations.text_level.ttr,measures.andrew.counts.acl,measures.andrew.counts.acl_ratio,measures.andrew.counts.ADJ,measures.andrew.counts.ADJ_ratio,...,measures.andrew.taassc.L2SCA.MLS,measures.andrew.taassc.L2SCA.MLT,measures.andrew.taassc.L2SCA.T_S,measures.andrew.taassc.L2SCA.VP_T,cleaned_text,sentences,measures.basic.n_sentences,tokens_per_sentence,measures.basic.total_n_tokens,measures.basic.avg_n_tokens_per_sentence
0,115499,b1,grandmas home remedies and recipes. do you hav...,German,0.108696,1.0,2,0.021739,3,0.032609,...,11.500000,11.500000,1.000000,1.500000,grandmas home remedies and recipes. do you hav...,"['grandmas home remedies and recipes.', 'do yo...",7,"[['grandmas', 'home', 'remedies', 'and', 'reci...",90,12.857143
1,1081381,a1,my friend is very nice.she comes from italy.sh...,Italian,0.111111,1.0,0,0.000000,6,0.166667,...,16.000000,5.333333,3.000000,1.000000,my friend is very nice.she comes from italy.sh...,['my friend is very nice.she comes from italy....,2,"[['my', 'friend', 'is', 'very', 'nice.she', 'c...",28,14.000000
2,452770,b1,first i will study a lot and finish my degree ...,Portuguese,0.176000,1.0,2,0.016000,5,0.040000,...,125.000000,13.888889,9.000000,1.777778,first i will study a lot and finish my degree ...,['first i will study a lot and finish my degre...,1,"[['first', 'i', 'will', 'study', 'a', 'lot', '...",113,113.000000
3,412035,a1,hy my names andr. i have thirty one years old....,Portuguese,0.050000,1.0,0,0.000000,2,0.050000,...,6.666667,10.000000,0.666667,1.500000,hy my names andr. i have thirty one years old....,"['hy my names andr.', 'i have thirty one years...",6,"[['hy', 'my', 'names', 'andr', '.'], ['i', 'ha...",36,6.000000
4,132380,b1,bello i glad to congratulate you with the best...,Russian,0.061224,1.0,0,0.000000,6,0.061224,...,8.636364,10.555556,0.818182,1.777778,bello i glad to congratulate you with the best...,['bello i glad to congratulate you with the be...,6,"[['bello', 'i', 'glad', 'to', 'congratulate', ...",84,14.000000


['measures.basic.n_sentences',
 'measures.basic.total_n_tokens',
 'measures.basic.avg_n_tokens_per_sentence']

In [19]:

# Create different classifiers.
classifiers = {
    "L2 logistic (Multinomial)": LogisticRegression(
        C=1, penalty="l2", solver="saga", max_iter=10000
    ),
    #"L2 logistic (OvR)": OneVsRestClassifier(
    #    LogisticRegression(C=C, penalty="l2", solver="saga", max_iter=10000)
    #)
}

In [20]:
features = {
    "n_sentences": ["measures.basic.n_sentences"],
    "n_sent+n_tokens": ["measures.basic.n_sentences","measures.basic.total_n_tokens"],
    #"avg-sent+n-sent+n-tokens": ["measures.basic.n_sentences","measures.basic.total_n_tokens","measures.basic.avg_n_tokens_per_sentence"],
    #"measures.basic": [c for c in andrew100k_train_fe_df.columns if "measures.basic" in c]
    #"measures.andrew": [c for c in andrew100k_train_fe_df.columns if "measures.andrew" in c]

}

In [22]:
target_column = config["column_mapping"]["andrew100k"]["CEFR"]
cv_results = defaultdict(lambda: defaultdict(dict))
n_epochs = 1
n_cv_folds = 10
for epoch_idx in range(n_epochs):
    dataset = andrew100k_train_fe_df.sample(frac=1)
    pipes = product(classifiers.values(), features.values())
    for model, feature_list in pipes:
        columns = feature_list+[target_column]
        kf = KFold(n_splits=n_cv_folds)
        model_idx = str(model)+str(feature_list)
        for cv_idx,(train_idx, test_idx) in enumerate(kf.split(dataset)):
            X_train = dataset.iloc[train_idx][feature_list]
            X_test  = dataset.iloc[test_idx][feature_list]
            y_train = dataset.iloc[train_idx][target_column]
            y_test  = dataset.iloc[test_idx][target_column]
            model.fit(X_train, y_train)
            target_names = ['A1', 'A2', 'B1', 'B2', 'C1']
            y_pred = model.predict(X_test)
            results_dict = classification_report(y_test, y_pred, target_names=target_names,output_dict=True)
            cv_results[model_idx]["accuracies"][str(epoch_idx)+str(cv_idx)] = results_dict["accuracy"]
            cv_results[model_idx]["results"][str(epoch_idx)+str(cv_idx)] = results_dict
            print(model_idx, results_dict["accuracy"])



LogisticRegression(C=1, max_iter=10000, solver='saga')['measures.basic.n_sentences'] 0.51325
LogisticRegression(C=1, max_iter=10000, solver='saga')['measures.basic.n_sentences'] 0.516875
LogisticRegression(C=1, max_iter=10000, solver='saga')['measures.basic.n_sentences'] 0.52225
LogisticRegression(C=1, max_iter=10000, solver='saga')['measures.basic.n_sentences'] 0.521875
LogisticRegression(C=1, max_iter=10000, solver='saga')['measures.basic.n_sentences'] 0.519125
LogisticRegression(C=1, max_iter=10000, solver='saga')['measures.basic.n_sentences'] 0.51925
LogisticRegression(C=1, max_iter=10000, solver='saga')['measures.basic.n_sentences'] 0.51775
LogisticRegression(C=1, max_iter=10000, solver='saga')['measures.basic.n_sentences'] 0.52125
LogisticRegression(C=1, max_iter=10000, solver='saga')['measures.basic.n_sentences'] 0.5070633829228653
LogisticRegression(C=1, max_iter=10000, solver='saga')['measures.basic.n_sentences'] 0.5234404300537567
LogisticRegression(C=1, max_iter=10000, solve

In [ ]:
df.iloc[[2,4,10]][["L1"]]

,L1
2,French
4,French
10,French


In [23]:
for strat, data in cv_results.items():
    print(strat, sum(data["accuracies"].values())/len(data["accuracies"].values()))

LogisticRegression(C=1, max_iter=10000, solver='saga')['measures.basic.n_sentences'] 0.5182128812976623
LogisticRegression(C=1, max_iter=10000, solver='saga')['measures.basic.n_sentences', 'measures.basic.total_n_tokens'] 0.7192803944243031


In [31]:
output_fp = "/datasets/NLP4CALL_2025_experiment/experiments_data/cv_results.pickle"
with open(output_fp,"wb") as outf:
  pickle.dump({k: v for k, v in cv_results.items()}, outf)

In [38]:
if remote_flags["output"] == False:
    pass
    #config["train_output_fp"] =  os.path.join(local_experiment_data_folder, "andrew100k-train-fe.csv")
    #config["test_output_fp"] =  os.path.join(local_experiment_data_folder, "andrew100k-test-fe.csv")
elif remote_flags["output"] == True:
    rclone.copyto(output_fp, config["cv_train_output_fp"])

Output()